In [3]:
!pip install pyngrok
!pip install flask-ngrok
!pip install pandas scikit-learn

# Import libraries
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

# Initialize Flask app
app = Flask(__name__)
ngrok.set_auth_token("NGROK_AUTHTOKEN") #replace with your authtoken code
# Create ngrok tunnel
public_url = ngrok.connect(5000)  # Port 5000 (default for Flask)
print(f"Public URL: {public_url}")

@app.route('/')
def home():
    return "Welcome to the Manufacturing API!"

# Global variables to store data and model
data = None
model1 = None
scaler = None

# Upload Endpoint
import os
@app.route('/upload', methods=['POST'])
def upload():
        global data
        try:
            print("Request received")
            file = request.files.get('File')  # Use get() for safety

            if file is None:
                print("File not found in request")
                return jsonify({"error": "File not found in request"}), 400

            print("File received:", file.filename)
            file_path = os.path.join('/content/', file.filename)

            try:
                file.save(file_path)
                print("File saved:", file_path)
            except Exception as save_error:
                print("Error saving file:", save_error)
                return jsonify({"error": f"Error saving file: {save_error}"}), 400

            try:
                data = pd.read_csv(file_path)
                print("File read successfully")
                return jsonify({"message": "File uploaded successfully!", "columns": list(data.columns)})
            except Exception as read_error:
                print("Error reading file:", read_error)
                return jsonify({"error": f"Error reading file: {read_error}"}), 400

        except Exception as e:
            print("General error:", e)
            return jsonify({"error": str(e)}), 400

# Train Endpoint
@app.route('/train', methods=['POST'])
def train():
    global model1, data, scaler
    if data is None:
        return jsonify({"error": "No data uploaded. Please upload data first."}), 400

    if scaler is None:
        scaler = MinMaxScaler()

    # Ensure required columns are in the dataset
    required_columns = ['Date', 'Machine_ID', 'Assembly_Line_No', 'Hydraulic_Pressure(bar)',
       'Coolant_Pressure(bar)', 'Air_System_Pressure(bar)',
       'Coolant_Temperature', 'Hydraulic_Oil_Temperature(?C)',
       'Spindle_Bearing_Temperature(?C)', 'Spindle_Vibration(?m)',
       'Tool_Vibration(?m)', 'Spindle_Speed(RPM)', 'Voltage(volts)',
       'Torque(Nm)', 'Cutting(kN)', 'Downtime']
    if not all(col in data.columns for col in required_columns):
        return jsonify({"error": f"Dataset must contain columns: {required_columns}"}), 400

    # Split data into features and target
    df = pd.DataFrame(data)
    df.drop(columns=['Date','Machine_ID','Assembly_Line_No'],axis=1,inplace=True)
    df.dropna(inplace=True)
    df.loc[df['Downtime']=='Machine_Failure','Downtime']=1
    df.loc[df['Downtime']=='No_Machine_Failure','Downtime']=0
    df['Downtime'] = pd.to_numeric(df['Downtime'])
    X=df.drop('Downtime',axis=1)
    y=df['Downtime']

    # Split into training and testing sets
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)

    #scaling
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the model
    regressor=LogisticRegression()
    model=regressor.fit(X_train,y_train)

    #Tune the model
    params={'C':[1,5,10,15,20,30,40,50],'max_iter':[100,150,200,300]}
    model1=GridSearchCV(model,param_grid=params,scoring='f1',cv=5)
    model1.fit(X_train,y_train)

    # Evaluate the model
    y_pred=model1.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return jsonify({"message": "Model trained successfully!", "accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1})

# Predict Endpoint
@app.route('/predict', methods=['POST'])
def predict():
    global model1, scaler
    if model1 is None:
        return jsonify({"error": "Model not trained. Please train the model first."}), 400

    try:
        # Get JSON input
        input_data = request.get_json()
        X_new = [[input_data["Hydraulic_Pressure(bar)"],input_data["Coolant_Pressure(bar)"],
                  input_data["Air_System_Pressure(bar)"],input_data["Coolant_Temperature"],input_data["Hydraulic_Oil_Temperature(?C)"],input_data["Spindle_Bearing_Temperature(?C)"],input_data["Spindle_Vibration(?m)"],
                  input_data["Tool_Vibration(?m)"],input_data["Spindle_Speed(RPM)"],input_data["Voltage(volts)"],input_data["Torque(Nm)"],
                  input_data["Cutting(kN)"]]]

        X_new = scaler.transform(X_new)

        # Make a prediction
        prediction = model1.predict(X_new)[0]
        decision_values = model1.decision_function(X_new)
        confidence = abs(decision_values[0])

        return jsonify({"Downtime": "Yes" if prediction == 1 else "No", "Confidence": round(confidence, 2)})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Run the Flask app
if __name__ == '__main__':
    app.run()


Public URL: NgrokTunnel: "https://5d74-34-21-0-180.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 18:46:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 18:46:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 18:46:26] "GET / HTTP/1.1" 200 -


Request received


INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 18:46:45] "POST /upload HTTP/1.1" 200 -


File received: Machine Downtime.csv
File saved: /content/Machine Downtime.csv
File read successfully


INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 18:46:59] "POST /train HTTP/1.1" 200 -
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 18:47:34] "POST /predict HTTP/1.1" 200 -
